In [17]:
import json
from openai import OpenAI
from dataclasses import dataclass, field
from typing import List, Callable, Any, Union
from enum import Enum
from pprint import pprint

In [2]:

# Setup OpenAI client to use Ollama
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

In [3]:

class StepType(Enum):
    """Types of steps in the execution loop"""
    CONTINUE = "continue"
    FINAL_OUTPUT = "final_output"
    HANDOFF = "handoff"
    MAX_TURNS = "max_turns"

@dataclass
class ExecutionStep:
    """Represents one step in the execution loop"""
    step_type: StepType
    agent_name: str
    content: str
    tool_calls: List[dict] = field(default_factory=list)
    next_agent: 'Agent' = None

In [4]:
@dataclass
class Tool:
    """Tool class"""
    name: str
    function: Callable
    parameters: dict

@dataclass
class Agent:
    """Agent class with tools and handoffs"""
    name: str
    instructions: str
    tools: List[Tool] = field(default_factory=list)
    handoffs: List['Agent'] = field(default_factory=list)
    model: str = "qwen3:8b"


In [33]:
from typing import List
from pprint import pprint
import json

class ExecutionLoop:
    """The core execution loop (similar to agents.Runner internals)"""

    def __init__(self):
        self.steps: List[ExecutionStep] = []

    def run(self, agent: Agent, input: str) -> str:
        """Main execution loop (no max_turns, runs until final output or handoff ends loop)"""
        print("🔄 EXECUTION LOOP STARTED")
        print("=" * 50)

        messages = [
            {"role": "system", "content": agent.instructions},
            {"role": "user", "content": input}
        ]

        current_agent = agent
        turn = 0

        while True:
            turn += 1
            print(f"\n📍 TURN {turn}")
            print(f"Current Agent: {current_agent.name}")

            # Step 1: Prepare tools and handoffs
            all_tools = self._prepare_tools(current_agent) or []
            print(f"Available tools: {len(all_tools)}")

            # Step 2: Call LLM
            print("🤖 Calling LLM...")
            response = client.chat.completions.create(
                model=current_agent.model,
                messages=messages,
                tools=all_tools if all_tools else None
            )

            message = response.choices[0].message

            print("--" * 40)
            pprint(message.model_dump())
            print("--" * 40)

            # Step 3: Process response
            if message.tool_calls:
                print(f"🔧 Tool calls detected: {len(message.tool_calls)}")

                # Add assistant message
                messages.append({
                    "role": "assistant",
                    "content": message.content,
                    "tool_calls": [
                        {
                            "id": tc.id,
                            "type": tc.type,
                            "function": {
                                "name": tc.function.name,
                                "arguments": tc.function.arguments
                            }
                        }
                        for tc in message.tool_calls
                    ]
                })

                # Step 4: Execute tools or handoffs
                step_result = self._execute_tools_and_handoffs(
                    current_agent, message.tool_calls, messages
                )

                self.steps.append(step_result)

                # Step 5: Check for handoff
                if step_result.step_type == StepType.HANDOFF:
                    print(f"🔄 Handoff: {current_agent.name} → {step_result.next_agent.name}")
                    current_agent = step_result.next_agent
                    messages[0] = {"role": "system", "content": current_agent.instructions}
                    continue

            else:
                # Step 6: Final output
                print("✅ Final output generated")
                final_step = ExecutionStep(
                    step_type=StepType.FINAL_OUTPUT,
                    agent_name=current_agent.name,
                    content=message.content
                )
                self.steps.append(final_step)

                self._print_execution_summary()
                return message.content

    def _prepare_tools(self, agent: Agent) -> List[dict]:
        """Prepare tools for OpenAI format"""
        tools = []

        # Add regular tools
        for tool in agent.tools:
            tools.append(tool.parameters)

        # Add handoff tools
        for handoff_agent in agent.handoffs:
            tools.append({
                "type": "function",
                "function": {
                    "name": f"handoff_to_{handoff_agent.name.lower().replace(' ', '_')}",
                    "description": f"Transfer to {handoff_agent.name}",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "reason": {"type": "string", "description": "Reason for handoff"}
                        },
                        "required": ["reason"]
                    }
                }
            })

        return tools

    def _execute_tools_and_handoffs(self, agent: Agent, tool_calls, messages) -> ExecutionStep:
        """Execute all tools and optionally perform one handoff"""
        handoff_step = None

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            print(f"  Executing: {function_name}({arguments})")

            # Check for handoff
            if function_name.startswith("handoff_to_") and handoff_step is None:
                target_name = function_name.replace("handoff_to_", "").replace("_", " ").title()

                for handoff_agent in agent.handoffs:
                    if handoff_agent.name.lower().replace(" ", "_") == target_name.lower().replace(" ", "_"):
                        messages.append({
                            "role": "tool",
                            "content": f"Transferred to {handoff_agent.name}",
                            "tool_call_id": tool_call.id
                        })

                        handoff_step = ExecutionStep(
                            step_type=StepType.HANDOFF,
                            agent_name=agent.name,
                            content=f"Handoff to {handoff_agent.name}",
                            next_agent=handoff_agent
                        )
                        break

            else:
                # Execute regular tool
                result = "Tool not found"
                for tool in agent.tools:
                    if tool.name == function_name:
                        try:
                            result = tool.function(**arguments)
                        except Exception as e:
                            result = f"Error: {e}"
                        break

                print(f"  Result: {result}")
                messages.append({
                    "role": "tool",
                    "content": str(result),
                    "tool_call_id": tool_call.id
                })

        return handoff_step or ExecutionStep(
            step_type=StepType.CONTINUE,
            agent_name=agent.name,
            content="All tools executed, continuing..."
        )

    def _print_execution_summary(self):
        """Print execution summary"""
        print("\n" + "=" * 50)
        print("🔍 EXECUTION SUMMARY")
        print("=" * 50)

        for i, step in enumerate(self.steps, 1):
            print(f"{i}. {step.step_type.value.upper()}: {step.agent_name}")
            print(f"   {step.content[:100]}...")
            if step.next_agent:
                print(f"   → Next: {step.next_agent.name}")


# ==== Demo Tools (unchanged) ====

def get_weather(city: str) -> str:
    return f"Weather in {city}: Sunny, 75°F"

def calculate(expression: str) -> str:
    try:
        return f"Result: {eval(expression)}"
    except:
        return "Invalid expression"

weather_tool = Tool(
    name="get_weather",
    function=get_weather,
    parameters={
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"]
            }
        }
    }
)

math_tool = Tool(
    name="calculate",
    function=calculate,
    parameters={
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Calculate",
            "parameters": {
                "type": "object",
                "properties": {"expression": {"type": "string"}},
                "required": ["expression"]
            }
        }
    }
)


In [34]:
# Create agents
weather_agent = Agent(
    name="Weather Agent",
    instructions="You are a weather specialist.",
    tools=[weather_tool]
)

math_agent = Agent(
    name="Math Agent", 
    instructions="You are a math specialist.",
    tools=[math_tool]
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="Route requests to specialists. For weather, handoff to Weather Agent. For math, handoff to Math Agent.",
    handoffs=[weather_agent, math_agent]
)

In [35]:
# Test execution loop
loop = ExecutionLoop()
result = loop.run(triage_agent, "What's the weather in Tokyo and calculate (15*30) + 25?")
print(f"\nFinal Result: {result}")


🔄 EXECUTION LOOP STARTED

📍 TURN 1
Current Agent: Triage Agent
Available tools: 2
🤖 Calling LLM...
--------------------------------------------------------------------------------
{'annotations': None,
 'audio': None,
 'content': '<think>\n'
            "Okay, let's tackle this user query. The user is asking for two "
            'things: the weather in Tokyo and the result of calculating '
            '(15*30) + 25. \n'
            '\n'
            'First, I need to determine if I should hand off each part to a '
            'specialist. The first part is about the weather, which matches '
            "the Weather Agent's function. The second part is a math "
            "calculation, which fits the Math Agent's function. \n"
            '\n'
            'For the weather part, the reason for handoff would be "requesting '
            'weather information for Tokyo". Then, for the math calculation, '
            'the reason would be "needing to calculate (15*30) + 25". \n'
            